In [3]:
import requests
import re
import urllib3
import csv
import time
import pandas as pd
from bs4 import BeautifulSoup
import re, json, csv, time, random
from urllib.parse import urlparse, parse_qs

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)


**Get url from salesnavigator**


In [6]:
session = '8585670718595733493'
at = 'AQEDAQuoAOQEd1Q0AAABhXStgN0AAAGFmLoE3U4Ahvg7TOKhV9g_z8PtrKN0zeaBR5HVuzeQmIYjR-F0yPnPQTuJe1DrQGB-H7Zj5O8uILSb-km9sK0bBsJ9WFbIV8Fq4-tIGqyUC63GgtbC8yOD7ZU0'
a = 'AQJ2PTEmc2FsZXNfY2lkPTExNjMyNTk2MDclM0ElM0E0NjIxOTExMDcVXy3ZmQFEg26YDcnGuU1Ee3JQ5g'

i=1
while i <=100:
    cookies = {
        'JSESSIONID': f'"ajax:{session}"',
        'liap': 'true',
        'li_at': at,
        'li_a': a,
        'lang': 'v=2&lang=es-es',
    }

    params={
            "q": "savedSearchId",
            "start": i,
            "count": "25",
            "savedSearchId": "50571251",
            "decorationId": "com.linkedin.sales.deco.desktop.searchv2.LeadSearchResult-9",
    }

    headers = {
        'sec-ch-ua': '"Not?A_Brand";v="8", "Chromium";v="108", "Google Chrome";v="108"',
        'x-li-lang': 'es_ES',
        'sec-ch-ua-mobile': '?0',
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36',
        'x-li-page-instance': 'urn:li:page:d_sales2_search_people_saved_all;ikwPoSXoQRuckSpkIxPUVQ==',
        'csrf-token': f'ajax:{session}',
        'x-restli-protocol-version': '2.0.0',
        'sec-ch-ua-platform': '"macOS"',
        'Accept': '*/*',
        'Sec-Fetch-Site': 'same-origin',
        'Sec-Fetch-Mode': 'cors',
        'Sec-Fetch-Dest': 'empty',
        'Accept-Language': 'en-US,en;q=0.9',
        'Host': 'www.linkedin.com',
        'Connection': 'close',
    }

    response = requests.get(
        'https://www.linkedin.com/sales-api/salesApiLeadSearch',
        cookies=cookies,
        headers=headers,
        params=params,
        verify=False
    )
    print(i,response.json().keys())
    # %%
    elements=response.json()['elements']
    profile_data=[]
    for profile in elements:
        profile_data+=[{'entityUrn' : profile['entityUrn']
        }]
    data_search = []
    for profile_data in profile_data:
        id=profile_data['entityUrn']
        data_search.append(["https://www.linkedin.com/in/"+re.search(r'\((.*?),', id).group(1), i])
    with open('linkedin_profile.csv', 'a') as f_object:
        writer_object = csv.writer(f_object)
        writer_object.writerows(data_search)
        f_object.close()
    print(f"in request i = {i}")
    time.sleep(3)
    i+=25

1 dict_keys(['metadata', 'elements', 'paging'])
in request i = 1
26 dict_keys(['metadata', 'elements', 'paging'])
in request i = 26
51 dict_keys(['metadata', 'elements', 'paging'])
in request i = 51
76 dict_keys(['metadata', 'elements', 'paging'])
in request i = 76


**Get publi Url from linkdin**


In [6]:
#%%


start_at = 884

def get_full_url(input_url):

    proxies = {
            'http': 'http://localhost:9090',
            'https': 'http://localhost:9090',
        }

    cookies = {
        'li_at': 'AQEDAQuoAOQACCMaAAABhXUbMH4AAAGFmSe0fk4AMO3VM32CYoSL2BsfPMALfM5-BBo902AeClLi9JttqzKcW8anZkgdQCB-He0R5c5-T2vZoplWLyPQTTu0-rTD0Xi50O0PpExVYNNBRj0JTf45Ix1m',
    }

    headers = {
        'Host': 'www.linkedin.com',
        'Connection': 'keep-alive',
        'Upgrade-Insecure-Requests': '1',
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
        'Sec-Fetch-Site': 'none',
        'Sec-Fetch-Mode': 'navigate',
        'Sec-Fetch-User': '?1',
        'Sec-Fetch-Dest': 'document',
        'Accept-Language': 'en-US,en;q=0.9',
    }

    # input_url = 'https://www.linkedin.com/in/ACwAAAQML5wBUcrTsc3b449Jy7x8SP6O-Fed3ok'
    response = requests.get(
        input_url,
        cookies=cookies,
        headers=headers,
        proxies=proxies,
        verify=False)

    soup = response.text

    with open("output1.html", "w") as file:
        file.write(str(soup))

    soup_ = BeautifulSoup(soup)
    json_code=json.loads(soup_.findAll('code', id=re.compile('^datalet-bpr-guid-'))[-1].text)
    json_code['request']

    parse_result = urlparse(json_code['request'])
    dict_result = parse_qs(parse_result.query)
    lk_id = dict_result['memberIdentity'][0]
    return lk_id

#%%

df = pd.read_csv('linkedin_profile.csv',)
input_url_list=df['url'].values

i=start_at
for url in input_url_list[start_at:]:
    data_search=get_full_url(url)
    with open('linkedin_profile_clean.csv', 'a') as f_object:
        writer_object = csv.writer(f_object)
        writer_object.writerow([data_search])
        f_object.close()
    i=i+1
    print(f"in request i = {i}")
    time.sleep(random.randrange(1,1000)/1000)

TooManyRedirects: Exceeded 30 redirects.